In [3]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot as hvp
import datetime
import altair as alt

In [4]:
# Creating a range of dates
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='D')
# Dates df
dates_df = pd.DataFrame(date_range, columns=['Date'])
#print(dates_df.head(1825))

# Creating a range of random "prices"
portfolio_df = pd.DataFrame(
    {
    "fake1": np.random.uniform(1000, -500, 100),
    "fake2": np.random.uniform(1000, -500, 100),
    "fake3": np.random.uniform(1000, -500, 100)
    }
    )
f1_portfolio_df = portfolio_df["fake1"].to_frame()

# Concatinating and finalizing test data
returns = pd.concat([dates_df, portfolio_df], join = "inner", axis = 1)
returns.set_index('Date', inplace = True)
returns = pd.DataFrame(returns)
returns.head()

,fake1,fake2,fake3
Date,,,
2018-01-01,370.17,997.76,-211.71
2018-01-02,763.05,744.16,233.63
2018-01-03,-309.44,-150.84,600.00
2018-01-04,-382.87,826.67,-9.61
2018-01-05,820.37,879.06,330.41


In [5]:
initial_investment = 10000

In [6]:
def cumulative_returns(df):
    return (1 + df).cumprod()

In [7]:
user_cr = cumulative_returns(portfolio_df)
user_cr

,fake1,fake2,fake3
0,371.17,998.76,-210.71
1,283592.41,744240.09,-49439.06
2,-87471435.53,-111514013.73,-29712995.88
3,33402604072.22,-92296898124.74,255816229.59
4,27435945665333.17,-81227070253938.64,84779742498.47
...,...,...,...
95,-1592016001213159931954983181141855848868652821...,-6669080194371434966975730247171745401847413941...,14706076377707886621587538250541780886110884704...
96,49588840910949677049993861773536274467985199595...,39481285134700637463323087831683346775080335967...,28700250777757478130277447259986741899714872607...
97,19100803369774009632943163044723884328131699310...,10922014003608890118700346949879142769058558217...,-1428655581377341982584071078762253070886138579...
98,97778116717738739991800194398393341203067184912...,-4017416600698394882035483092596977882066063885...,36411613391527270847628741704679084907131102585...


## Correlation

In [10]:
returns_cor = returns.corr()
returns_cor

,fake1,fake2,fake3
fake1,1.00,0.17,-0.07
fake2,0.17,1.00,-0.09
fake3,-0.07,-0.09,1.00


## Correlation Plot

In [12]:
#In the works
# source = returns_cor

# alt.Chart(source).mark_circle(size=60).encode(
#     x='Horsepower',
#     y='Miles_per_Gallon',
#     color='Origin',
#     tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
# ).interactive()

In [ ]:
def covariance(df, ticker, market):
    return df[ticker].cov(df[market])

In [ ]:
user_covariance = covariance(portfolio_df, 'fake2', 'fake3')
user_covariance

In [ ]:
def variance(df, market):
    return df[market].var()

In [ ]:
user_variance = variance(portfolio_df, 'fake3')
user_variance

### Drawdown

In [ ]:
def drawdown(df, tickers):
    Roll_Max = df[tickers].cummax()
    Daily_Drawdown = df[tickers]/Roll_Max - 1.0
    Max_Daily_Drawdown = Daily_Drawdown.cummin() * 100

In [ ]:
user_drawdown = drawdown(portfolio_df, 'fake1')
user_drawdown

### Drawdown Chart

In [ ]:
def drawdown_chart(returns, date, quant):
    '''
    return: is a dataframe
    date: str: should be a date column name
    quant: str: should be a quantitative data type
    '''
    chart = alt.Chart(df.reset_index()).mark_bar().encode(
        x = date + ':T',  # T specifies temporal data type for x-axis
        y = quant + ':Q',  # Q specifies a quantitative data type for y-axis
        color = alt.condition(
            getattr(alt.datum, quant) > 0,
            alt.value("Green"),  # The positive color
            alt.value("red")
    ), tooltip= [date + ':T', quant + ':Q']
    ).properties(width=800)
    chart = chart.interactive()
    return chart

In [ ]:
def tracking_error(df, tickers, market):
    track_error = np.sqrt(sum([i**2 for i in df[tickers] - df[market]]))
    return track_error

In [ ]:
user_tracking_error = tracking_error(portfolio_df, 'fake2', 'fake3')
user_tracking_error

In [ ]:
def beta(covariance, variance):
    chart_beta = covariance / variance
    chart_beta = chart_beta.rolling(window = 21).plot()
    plt.show()
    beta.shape
    chart_beta.plot()
    return chart_beta

In [ ]:
# user_beta = beta(user_covariance, user_variance)
# user_beta
returns.tail()

,fake1,fake2,fake3
Date,,,
2018-04-06,907.08,727.80,-258.16
2018-04-07,450.55,926.38,816.60
2018-04-08,183.61,-466.01,176.75
2018-04-09,675.85,445.23,593.01
2018-04-10,917.91,498.30,877.10


In [ ]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

In [ ]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
print(user_sharpe_ratio)

## Return on Investment

In [ ]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    chart_return_oi = return_oi.plot()
    plt.show()
    return(return_oi)

## ROI chart

In [ ]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    chart_return_oi = return_oi
    plt.show()
    return(return_oi)

In [ ]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi

,fake1,fake2,fake3
0,-480.82,-494.03,350.46
1,57835.76,-134065.92,176570.66
2,28650118.88,57699501.32,131842849.53
3,3617819587.11,57218554741.80,-48305185652.85
4,-257528051241.47,-27244507281509.60,-27740192623146.52
...,...,...,...
95,-7972071981440524776547539071537770031039396053...,-4743105153601428239207943572442221754097997037...,-3012469811436924757166351733881725817985390397...
96,-6035694403311019687938991770667577575126963207...,-2912643707175354029636790484322618363320270248...,11268391418462026261230672997052966158129016405...
97,11332110595457437722363591512688828514783400198...,-2413123043331214749784477421943330954067332986...,71443476518462852478681769246839488639949623178...
98,-3161384451529218882238401070351922761500642363...,-2337129121272126681852557161865977775103106180...,16729057682066967710303856286047031312443298882...


source = data.wheat()

bar = alt.Chart(source).mark_bar().encode(
    x = df + ':O',
    y = returns + ':Q'
)

<!-- line = alt.Chart(source).mark_line(color='red').transform_window(
    # The field to average
    rolling_mean='mean(wheat)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='year:O',
    y='rolling_mean:Q'
)

(bar + line).properties(width=600) -->

   ## Porfolio Size and Holdings

In [13]:
# In the works
# source = returns

# alt.Chart(source).mark_arc().encode(
#     theta="value",
#     color = returns
# )

# HI MOM MUSTANGS ARE FUEGO